# Scrape Seeking Alpha

- Seeking Alpha compiles Earnings Call transcripts
- scraping requires a seeking alpha account

In [48]:
# Jupyter magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
# imports
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import pandas as pd
from seleniumrequests import Chrome
import pathlib

In [50]:
save_dir = "../1_data/Seeking_Alpha"
# pathlib.Path(save_dir).mkdir()
pathlib.Path(save_dir).exists()

True

## Create index of URLs for earnings call transcripts

## Setup the Selenium Window for Querying

This part requries downloading a driver for selenium. I used Chrome to get this running.


In [16]:
!pwd

/Users/ahale/Projects/a-list/notebooks


In [51]:
# Change this to where the chromedriver is in your environment
path_to_chromedriver = '/Users/ahale/chromedriver'

## pull the data and write a csv for an index

In [53]:
# list_item_df = pd.DataFrame()

In [143]:
# Get index of all articles
# Need to do this iteratively so that Seeking Alpha doesn't catch on

driver = Chrome(executable_path=path_to_chromedriver)
for i in range(1,2):
    url = "https://seekingalpha.com/earnings/earnings-call-transcripts/" + str(i)
    display(url)
#     driver = Chrome(executable_path=path_to_chromedriver)
    driver.get(url)
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    listitems= soup.findAll("li", {"class": "list-group-item article"})

    for listitem in listitems:
        link = listitem.find("a", {"class": "dashboard-article-link"})
        title = link.text
        article_url = link.get('href')

        desc = listitem.find('div')
        org = desc.find('span').find('a').get('title')
        date = re.search('•\n      (.*)\n\n        •', desc.text).group(1)
        # Add metadata to index
        list_item_df = list_item_df.append({
            'title': title
            , 'url': 'https://seekingalpha.com' + article_url
            , 'org': org
            , 'date': date
        }, ignore_index=True)
    time.sleep(random.random())
    
    # Close the chrome window so I don't end up with 1000s open on my machine
#     driver.close()
list_item_df

'https://seekingalpha.com/earnings/earnings-call-transcripts/1598'

'https://seekingalpha.com/earnings/earnings-call-transcripts/1599'

'https://seekingalpha.com/earnings/earnings-call-transcripts/1600'

'https://seekingalpha.com/earnings/earnings-call-transcripts/1601'

'https://seekingalpha.com/earnings/earnings-call-transcripts/1602'

...

'https://seekingalpha.com/earnings/earnings-call-transcripts/1796'

'https://seekingalpha.com/earnings/earnings-call-transcripts/1797'

'https://seekingalpha.com/earnings/earnings-call-transcripts/1798'

'https://seekingalpha.com/earnings/earnings-call-transcripts/1799'

,date,org,title,url
0,"Fri, Nov. 20, 10:32 PM","Helmerich & Payne, Inc.","Helmerich & Payne, Inc. (HP) CEO John Lindsay ...",https://seekingalpha.com/article/4390777-helme...
1,"Fri, Nov. 20, 9:54 PM",GSX Techedu Inc.,GSX Techedu's (GSX) CEO Larry Chen on Q3 2020 ...,https://seekingalpha.com/article/4390776-gsx-t...
2,"Fri, Nov. 20, 6:28 PM",Eli Lilly and Company,Eli Lilly and Company (LLY) Presents Tirzepati...,https://seekingalpha.com/article/4390771-eli-l...
3,"Fri, Nov. 20, 5:21 PM","Cresud Sociedad Anónima Comercial, Inmobiliari...",Cresud SACIF y A (CRESY) CEO Alejandro Elsztai...,https://seekingalpha.com/article/4390758-cresu...
4,"Fri, Nov. 20, 4:21 PM",IRSA Inversiones y Representaciones Sociedad A...,IRSA Inversiones y Representaciones Sociedad A...,https://seekingalpha.com/article/4390744-irsa-...
...,...,...,...,...
53965,"Nov. 2, 2017, 8:55 PM","Activision Blizzard, Inc.",Activision Blizzard (ATVI) Q3 2017 Results - E...,https://seekingalpha.com/article/4119881-activ...
53966,"Nov. 2, 2017, 8:51 PM","Bottomline Technologies (de), Inc.",Bottomline Technologies (EPAY) Q1 2018 Results...,https://seekingalpha.com/article/4119880-botto...
53967,"Nov. 2, 2017, 8:47 PM",Fluor Corporation,Fluor (FLR) Q3 2017 Results - Earnings Call Tr...,https://seekingalpha.com/article/4119878-fluor...
53968,"Nov. 2, 2017, 8:46 PM",Apple Inc.,Apple (AAPL) Q4 2017 Results - Earnings Call T...,https://seekingalpha.com/article/4119877-apple...


In [98]:
# list_item_df = list_item_df.drop(columns='Unnamed: 0')

In [149]:
# list_item_df.drop_duplicates().to_csv('../1_data/Seeking_Alpha/index.csv', index=False)

In [211]:
# Get the index dataframe given that it's already written
seeking_alpha_index = pd.read_csv('../1_data/Seeking_Alpha/index.csv')
seeking_alpha_index['filename'] = None
seeking_alpha_index.head()

,date,org,title,url,filename
0,"Fri, Nov. 20, 10:32 PM","Helmerich & Payne, Inc.","Helmerich & Payne, Inc. (HP) CEO John Lindsay ...",https://seekingalpha.com/article/4390777-helme...,None
1,"Fri, Nov. 20, 9:54 PM",GSX Techedu Inc.,GSX Techedu's (GSX) CEO Larry Chen on Q3 2020 ...,https://seekingalpha.com/article/4390776-gsx-t...,None
2,"Fri, Nov. 20, 6:28 PM",Eli Lilly and Company,Eli Lilly and Company (LLY) Presents Tirzepati...,https://seekingalpha.com/article/4390771-eli-l...,None
3,"Fri, Nov. 20, 5:21 PM","Cresud Sociedad Anónima Comercial, Inmobiliari...",Cresud SACIF y A (CRESY) CEO Alejandro Elsztai...,https://seekingalpha.com/article/4390758-cresu...,None
4,"Fri, Nov. 20, 4:21 PM",IRSA Inversiones y Representaciones Sociedad A...,IRSA Inversiones y Representaciones Sociedad A...,https://seekingalpha.com/article/4390744-irsa-...,None


In [182]:
# TODO: change index date to date
seeking_alpha_index.date.head()

0    Fri, Nov. 20, 10:32 PM
1    Fri, Nov. 20,  9:54 PM
2    Fri, Nov. 20,  6:28 PM
3    Fri, Nov. 20,  5:21 PM
4    Fri, Nov. 20,  4:21 PM
Name: date, dtype: object

### Get the text for each earnings transcript


In [ ]:
# Need to launch the driver once so that we can
driver = Chrome(executable_path=path_to_chromedriver)

In [226]:
# Loop through all transcripts in the index
save_dir = '../1_data/Seeking_Alpha/'
for index, row in seeking_alpha_index.iloc[20:40].iterrows():
    
    url = row.url
    display(url)
    driver.get(url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    listitems= soup.findAll("p")
    
    # Get the transcript text and save it as a file
    org_name = '-'.join([i for i in re.split('[^a-zA-Z0-9\n\.]', row.org.replace('.', '')) if i])
    date = '-'.join([i for i in re.split('[^a-zA-Z0-9\n\.]', row.date.replace('.', '')) if i])
    filename = org_name + '_' + date
    seeking_alpha_index.filename[index] = f'{filename}.txt'

    transcript_text = ''
    for item in listitems:
        transcript_text = transcript_text + ' ' + item.text

    fname = f"{save_dir}/{filename}.txt"
    with open(fname, 'w') as f:
        f.writelines(transcript_text)
    time.sleep(random.random())

'https://seekingalpha.com/article/4390645-matthews-international-corp-matw-ceo-joseph-bartolacci-on-q4-2020-results-earnings-call'

'https://seekingalpha.com/article/4390644-t-inc-t-presents-morgan-stanley-tmt-broker-conference-call-transcript'

'https://seekingalpha.com/article/4390634-i3-verticals-inc-iiiv-ceo-gregory-daily-on-q4-2020-results-earnings-call-transcript'

'https://seekingalpha.com/article/4390626-post-holdings-inc-post-ceo-rob-vitale-on-q4-2020-results-earnings-call-transcript'

'https://seekingalpha.com/article/4390582-ooma-inc-ooma-ceo-eric-stang-on-q3-2021-results-earnings-call-transcript'

...

'https://seekingalpha.com/article/4390514-delta-apparel-inc-dla-ceo-robert-humphreys-on-q4-2020-results-earnings-call-transcript'

'https://seekingalpha.com/article/4390512-woodward-inc-wwd-ceo-tom-gendron-on-q4-2020-results-earnings-call-transcript'

'https://seekingalpha.com/article/4390511-biohitech-global-inc-bhtg-ceo-anthony-fuller-on-q3-2020-results-earnings-call-transcript'

'https://seekingalpha.com/article/4390508-111-inc-yi-ceo-junling-liu-on-q3-2020-results-earnings-call-transcript'

'https://seekingalpha.com/article/4390507-sociedad-quimica-y-minera-de-chiles-sqm-ceo-ricardo-ramos-on-q3-2020-results-earnings-call'

In [220]:
seeking_alpha_index.to_csv('../1_data/Seeking_Alpha/index_2.csv')

In [227]:
seeking_alpha_index.iloc[0:50]

,date,org,title,url,filename
0,"Fri, Nov. 20, 10:32 PM","Helmerich & Payne, Inc.","Helmerich & Payne, Inc. (HP) CEO John Lindsay ...",https://seekingalpha.com/article/4390777-helme...,Helmerich-Payne-Inc_Fri-Nov-20-10-32-PM.txt
1,"Fri, Nov. 20, 9:54 PM",GSX Techedu Inc.,GSX Techedu's (GSX) CEO Larry Chen on Q3 2020 ...,https://seekingalpha.com/article/4390776-gsx-t...,GSX-Techedu-Inc_Fri-Nov-20-9-54-PM.txt
2,"Fri, Nov. 20, 6:28 PM",Eli Lilly and Company,Eli Lilly and Company (LLY) Presents Tirzepati...,https://seekingalpha.com/article/4390771-eli-l...,Eli-Lilly-and-Company_Fri-Nov-20-6-28-PM.txt
3,"Fri, Nov. 20, 5:21 PM","Cresud Sociedad Anónima Comercial, Inmobiliari...",Cresud SACIF y A (CRESY) CEO Alejandro Elsztai...,https://seekingalpha.com/article/4390758-cresu...,Cresud-Sociedad-An-nima-Comercial-Inmobiliaria...
4,"Fri, Nov. 20, 4:21 PM",IRSA Inversiones y Representaciones Sociedad A...,IRSA Inversiones y Representaciones Sociedad A...,https://seekingalpha.com/article/4390744-irsa-...,IRSA-Inversiones-y-Representaciones-Sociedad-A...
5,"Fri, Nov. 20, 4:01 PM","Magellan Midstream Partners, L.P.","Magellan Midstream Partners, L.P. (MMP) Presen...",https://seekingalpha.com/article/4390740-magel...,Magellan-Midstream-Partners-LP_Fri-Nov-20-4-01...
6,"Fri, Nov. 20, 3:50 PM",Grupo Supervielle S.A.,Grupo Supervielle's (SUPV) CEO Patricio Superv...,https://seekingalpha.com/article/4390737-grupo...,Grupo-Supervielle-SA_Fri-Nov-20-3-50-PM.txt
7,"Fri, Nov. 20, 3:35 PM",PennantPark Investment,PennantPark Investment Corporation (PNNT) CEO ...,https://seekingalpha.com/article/4390727-penna...,PennantPark-Investment_Fri-Nov-20-3-35-PM.txt
8,"Fri, Nov. 20, 3:31 PM",Geospace Technologies Corporation,Geospace Technologies Corporation (GEOS) CEO W...,https://seekingalpha.com/article/4390719-geosp...,Geospace-Technologies-Corporation_Fri-Nov-20-3...
9,"Fri, Nov. 20, 3:25 PM",The RMR Group Inc.,The RMR Group's (RMR) CEO Adam Portnoy on Q4 2...,https://seekingalpha.com/article/4390712-rmr-g...,The-RMR-Group-Inc_Fri-Nov-20-3-25-PM.txt
